In [26]:
!pip install gender-guesser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 379 kB 4.3 MB/s 


In [27]:
import gender_guesser.detector as EN_gender

In [2]:
import json
from collections import defaultdict, Counter

In [3]:
import pandas as pd
import ast

# Reformat

In [13]:
def reformat(path, df_main,country_to_select,year):

  print(country_to_select)

  df = pd.read_csv(path)
  print("Patents Dataset-",df.shape)

  df_filtered_list = []
  idx = 0

  for i in df.index:

    applications = ast.literal_eval(df.loc[i,"WorldWide Applications"])

    if applications==['No KR or CN Application Number'] or applications==['Patent not found'] :
      continue

    app = None
    app_index = None
    for j in range(len(applications)):
      if applications[j][0:2]==country_to_select:
        app = applications[j]
        app_index = j
    if app==None:
      continue

    inventor_countries = ast.literal_eval(df.loc[i,"inventor_country"])
    
    if country_to_select not in inventor_countries:
      continue

    original_names = ast.literal_eval(df.loc[i,"Original Names"])
    english_names = ast.literal_eval(df.loc[i,"inventor_name"])
    english_first_names = ast.literal_eval(df.loc[i,"inventor_firstname"])

    assert len(inventor_countries)==len(english_names) and len(english_names)==len(english_first_names)

    for k in range(len(inventor_countries)):
      if inventor_countries[k]==country_to_select:
        try:
          df_filtered_list.append([df.loc[i,"publication_id"],df.loc[i,"country"],english_names[k],english_first_names[k],\
                                      inventor_countries[k],app,original_names[app_index][k],year])
        except:
          pass
          #df_filtered_list.append([df.loc[i,"publication_id"],df.loc[i,"country"],english_names[k],english_first_names[k],\
          #                            inventor_countries[k],app,"",year])
      #idx+=1

    if (len(df_filtered_list)+df_main[df_main["year"]==year].shape[0])>=8000:
      break

  df_filtered = pd.DataFrame(df_filtered_list, columns = ["publication_id","country","english_names","english_first_names","inventor_country","worldwide_application","original_names","year"])
  
  df_main = pd.concat([df_main,df_filtered])

  df_main = df_main.reset_index(drop=True)
  print("Inventors Dataset-",df_main.shape)
  
  return df_main

In [14]:
df_main_CN = pd.DataFrame(columns = ["publication_id","country","english_names","english_first_names",\
                                  "inventor_country","worldwide_application","original_names","year"])
df_main_KR = pd.DataFrame(columns = ["publication_id","country","english_names","english_first_names",\
                                  "inventor_country","worldwide_application","original_names","year"])

In [15]:
path_list = ["/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/2017_CN_KR_patent_grant_processed_names_full.csv",\
             "/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/2018_CN_KR_patent_grant_processed_names_full.csv",\
             "/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/2019_CN_KR_patent_grant_processed_names.csv",\
             "/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/2020_CN_KR_patent_grant_processed_names.csv",\
             "/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/2021_CN_KR_patent_grant_processed_names_part1.csv",\
             "/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/2021_CN_KR_patent_grant_processed_names_part2.csv",\
            ]
year_list = [2017,2018,2019,2020,2021,2021]

In [16]:
a = 0
for p in path_list:
  print(year_list[a])
  df_main_CN = reformat(p, df_main_CN ,country_to_select="CN",year=year_list[a])
  df_main_KR = reformat(p, df_main_KR ,country_to_select="KR",year=year_list[a])
  a+=1

2017
CN
Patents Dataset- (38452, 23)
Inventors Dataset- (8003, 8)
KR
Patents Dataset- (38452, 23)
Inventors Dataset- (8000, 8)
2018
CN
Patents Dataset- (38768, 23)
Inventors Dataset- (16008, 8)
KR
Patents Dataset- (38768, 23)
Inventors Dataset- (16000, 8)
2019
CN
Patents Dataset- (50960, 23)
Inventors Dataset- (24011, 8)
KR
Patents Dataset- (50960, 23)
Inventors Dataset- (24001, 8)
2020
CN
Patents Dataset- (55183, 23)
Inventors Dataset- (32012, 8)
KR
Patents Dataset- (55183, 23)
Inventors Dataset- (32002, 8)
2021
CN
Patents Dataset- (10899, 23)
Inventors Dataset- (40012, 8)
KR
Patents Dataset- (10899, 23)
Inventors Dataset- (40004, 8)
2021
CN
Patents Dataset- (10699, 23)
Inventors Dataset- (40017, 8)
KR
Patents Dataset- (10699, 23)
Inventors Dataset- (40007, 8)


In [17]:
print(df_main_CN.shape)
df_main_CN.head()

(40017, 8)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year
0,US09532512,CN,Xie Yunlong,Yunlong,CN,CN104823754A/en,谢云龙,2017
1,US09532598,US,Liu Qiuming,Qiuming,CN,CN205214206U/en,刘秋明,2017
2,US09532599,CN,Liu Qiuming,Qiuming,CN,CN203327956U/en,刘秋明,2017
3,US09532602,CN,Liu Qiuming,Qiuming,CN,CN203378559U/en,刘秋明,2017
4,US09532631,WO,Gu Jiawei,Jiawei,CN,CN105378583B/en,顾嘉唯,2017


In [18]:
df_main_CN["year"].value_counts()

2018    8005
2021    8005
2017    8003
2019    8003
2020    8001
Name: year, dtype: int64

In [19]:
df_main_CN.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_CN_Names.csv",index=False)

In [20]:
print(df_main_KR.shape)
df_main_KR.head()

(40007, 8)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year
0,USD0775625,KR,Kim Dong-Hyun,Dong-Hyun,KR,KR101515629B1/en,김동현,2017
1,US09532634,KR,Lee Cheon Sook,Cheon Sook,KR,KR101469765B1/en,이영훈,2017
2,US09532636,KR,Jung Ku Youl,Ku Youl,KR,KR101318467B1/en,정규열,2017
3,US09532637,KR,Choi Jung Sun,Jung Sun,KR,KR20140094485A/en,최정선,2017
4,US09532637,KR,Kim Kyung Nam,Kyung Nam,KR,KR20140094485A/en,김경남,2017


In [21]:
df_main_KR["year"].value_counts()

2021    8005
2019    8001
2020    8001
2017    8000
2018    8000
Name: year, dtype: int64

In [22]:
df_main_KR.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names.csv",index=False)

# English Names Gender Guesser Function

In [23]:
gender_dict_full = pd.read_csv('/content/drive/MyDrive/Team 1/Data/wgnd_2_0_name-gender_nocode.csv', header=None, index_col=0, squeeze=True).to_dict()

In [24]:
gender_dict_kr = pd.read_csv('/content/drive/MyDrive/Team 1/Data/gender_dict_kr.csv', header=None, index_col=0, squeeze=True).to_dict()
gender_dict_cn = pd.read_csv('/content/drive/MyDrive/Team 1/Data/gender_dict_cn.csv', header=None, index_col=0, squeeze=True).to_dict()
print(len(gender_dict_full),len(gender_dict_cn),len(gender_dict_kr))

3491141 687111 8529


In [28]:
d = EN_gender.Detector()

def english_gender_CN(L):
  
  try:
    gender = gender_dict_cn[L]
  except:
    try:
      gender = gender_dict_full[L]
    except:
      gender = d.get_gender(L)
  if gender=="M":
    return "male"
  elif gender=="F":
    return "female"
  else:
    return gender

def english_gender_KR(L):
  
  try:
    gender = gender_dict_kr[L]
  except:
    try:
      gender = gender_dict_full[L]
    except:
      gender = d.get_gender(L)
  if gender=="M":
    return "male"
  elif gender=="F":
    return "female"
  else:
    return gender

# Original Chinese Names Gender Guesser Function

In [29]:
class Gender:
    '''Predict the gender(s) of (a) given name(s) in Chinese.
    
    Basic usage:
    =================================
    name = a name or a list of names
    gender = Gender()
    gender.predict(name)
    =================================
    
    Gender().predict()
    =================================
    Paras:
        name: str or list
        method: "lap" or "gt", defaults to "lap"
            lap --> adjusts the training set by laplace smoothing
            gt --> adjusts the training set by Good Turing smoothing
        show_all: bool, defaults to True
            True --> Returns the probablities for all genders (M, F, U)
            False --> Returns the predicted gender with optimal probablity
    =================================
    
    Notes:
    =================================
    The two smoothing methods assume unseen characters to add. The training set contains 
    about 5000 unique characters for M and F and the default number of unseen characters
    is set to be 5000, although it turns out to be very insignificant. 
    
    To reset this number, when calling the Gender class, make gender=Gender(your_num).
    '''
    def __init__(self, num_unseen_chars=5000):
        # num of unseen chars for Chinese names
        self._unseen = num_unseen_chars
        self.name = 'You have not entered a name yet'
        # loading the unsmoothed training set
        self.genderDict = self._loadDict()
        # laplace-adjusted genderDict
        self._lapDict = self._laplace()
        # frequency-based good-turing dict
        self._gtDict = self._goodTuring()
        
        # get last names for later splitting first names
        f = open('/content/drive/MyDrive/Team 1/Data/gender_predicator_main_Naive_Bayes_Gender/ChineseLastNames.txt', 'r', encoding="utf-8")
        next(f)
        lastnames = [line.split('\t')[0] for line in f]    
        uni_nam, bi_nam = (), ()
        for n in lastnames:
            if len(n) == 1: uni_nam += (n, )
            else: bi_nam += (n, )
        self._uni_nam = uni_nam
        self._bi_nam = bi_nam
        
    def _loadDict(self):
        genderDict = json.load(open('/content/drive/MyDrive/Team 1/Data/gender_predicator_main_Naive_Bayes_Gender/dict4Gender.json', 'r'))
        genderDict = {k: Counter(v) for k, v in genderDict.items()}
        return defaultdict(Counter, genderDict)
        
    
    def _laplace(self):
        '''Converts the dict into one suitable for laplace smoothing.
        '''
        lapDict = self.genderDict.copy()
        total = lapDict.pop('total')
        # number of unique chars used for each gender
        distinct = Counter([gender for v in lapDict.values() for gender in v.keys()])
        for g in ['M', 'F', 'U']:
            # add the estimated unseen chars (suppose 5000) to each 
            # gender category both in terms of distinct chars and total chars
            distinct[g] += self._unseen
            total[g] += distinct[g]
        lapDict['total'] = total
        return lapDict
        
    def _goodTuring(self):
        '''Rerturns a dict that contains the occurences info for each freq category
        '''
        genDict = self.genderDict.copy()
        total = genDict.pop('total')
        gtDict = defaultdict(Counter, {0: {'M': self._unseen, 'F': self._unseen, 'U': self._unseen}})
        for V in genDict.values():
            for k, v in V.items():
                gtDict[v][k] += 1
        return gtDict
    
    def _getFirstName(self, name):
        '''Returns the first name of a given name.
        '''
        if name.startswith(self._bi_nam): return name[2:]
        elif name.startswith(self._uni_nam): return name[1:]
        else: return name
        
    
    def _naiveBayesP(self, fname, gender, method='lap'):
        '''Returns the naive bayes probablity of a given gender for a given name.
        '''
        def getNr(r):
            idx = 0
            Nr = self._gtDict[r][gender]
            while not Nr:
                Nr = self._gtDict[r-idx][gender]
                idx += 1
            
            return Nr
        
        if method == 'lap':
            total_char = sum(self._lapDict['total'].values())
            gender_char = self._lapDict['total'][gender]
            p_gender =  gender_char / total_char
            for char in fname:
                char_dict = self._lapDict[char]
                p_char_g = (char_dict[gender] + 1) / gender_char
                p_gender *= p_char_g
            return p_gender
        elif method == 'gt':
            total_char = sum(self.genderDict['total'].values())
            gender_char = self.genderDict['total'][gender]
            p_gender =  gender_char / total_char
            for char in fname:
                char_dict = self.genderDict[char]
                r = char_dict[gender]
                Nr, NrPlus1 = getNr(r), getNr(r+1)
                r_adj = (r + 1) * NrPlus1 / Nr
                p_char_g = r_adj / gender_char
                p_gender *= p_char_g
            return p_gender
        else:
            raise ValueError(f'{method} not available. Please use\n'
            '"lap" --> for laplace-adjust prediction (default).\n'
            '"gt" --> for good-turing-adjusted prediction.')
                
    def predict(self, name, method='lap', show_all=True):
        '''Returns the probablities of genders for (a) given name(s).
        '''
        def run(fname, show_all):
            nonlocal res
            pM = self._naiveBayesP(fname, 'M', method=method)
            pF = self._naiveBayesP(fname, 'F', method=method)
            pU = self._naiveBayesP(fname, 'U', method=method)
            totalP = pM + pF + pU
            pM, pF, pU = pM/totalP, pF/totalP, pU/totalP
            if show_all:
                res.append((self.name, {'M': pM, 'F': pF, 'U': pU}))
            else:
                if pM==pF and pM==pU: res.append((self.name, 'M=F=Undefined', pM))
                elif pM == max(pM, pF, pU): res.append((self.name, 'M', pM))
                elif pF > pU: res.append((self.name, 'F', pF))
                else: res.append((self.name, 'Undefined', pU))
        
        res = []
        if isinstance(name, str):
            self.name = name
            fname = self._getFirstName(name)
            run(fname, show_all)
            return res[0]
        elif isinstance(name, list):
            for n in name:
                self.name = n
                fname = self._getFirstName(n)
                run(fname, show_all)
            return res
        else:
            raise TypeError('name must be either a str or a list')

In [30]:
CN_gender = Gender()

def chinese_gender(L):

  result = CN_gender.predict(L, show_all=False)

  return result[1]

def chinese_gender_prob(L):

  result = CN_gender.predict(L, show_all=False)

  return result[2]

# Guessing the gender for chinese names

In [31]:
df_main_CN_gender = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_CN_Names.csv")
print(df_main_CN_gender.shape)
df_main_CN_gender.head()

(40017, 8)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year
0,US09532512,CN,Xie Yunlong,Yunlong,CN,CN104823754A/en,谢云龙,2017
1,US09532598,US,Liu Qiuming,Qiuming,CN,CN205214206U/en,刘秋明,2017
2,US09532599,CN,Liu Qiuming,Qiuming,CN,CN203327956U/en,刘秋明,2017
3,US09532602,CN,Liu Qiuming,Qiuming,CN,CN203378559U/en,刘秋明,2017
4,US09532631,WO,Gu Jiawei,Jiawei,CN,CN105378583B/en,顾嘉唯,2017


In [32]:
df_main_CN_gender["english_gender"] = df_main_CN_gender["english_first_names"].apply(english_gender_CN)

In [33]:
df_main_CN_gender["english_gender"].value_counts()

unknown          24683
andy             12471
male              1467
mostly_male        849
female             443
mostly_female      104
Name: english_gender, dtype: int64

In [34]:
df_main_CN_gender["chinese_gender"] = df_main_CN_gender["original_names"].apply(chinese_gender)
df_main_CN_gender["chinese_gender_prob"] = df_main_CN_gender["original_names"].apply(chinese_gender_prob)

In [35]:
df_main_CN_gender["chinese_gender"].value_counts()

M                31866
F                 7208
Undefined          887
M=F=Undefined       56
Name: chinese_gender, dtype: int64

In [36]:
print(df_main_CN_gender.shape)
df_main_CN_gender.head()

(40017, 11)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,chinese_gender,chinese_gender_prob
0,US09532512,CN,Xie Yunlong,Yunlong,CN,CN104823754A/en,谢云龙,2017,unknown,M,0.945586
1,US09532598,US,Liu Qiuming,Qiuming,CN,CN205214206U/en,刘秋明,2017,unknown,M,0.500785
2,US09532599,CN,Liu Qiuming,Qiuming,CN,CN203327956U/en,刘秋明,2017,unknown,M,0.500785
3,US09532602,CN,Liu Qiuming,Qiuming,CN,CN203378559U/en,刘秋明,2017,unknown,M,0.500785
4,US09532631,WO,Gu Jiawei,Jiawei,CN,CN105378583B/en,顾嘉唯,2017,unknown,M,0.496414


In [37]:
df_main_CN_gender.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_CN_Names_Gender_CN_EN.csv",index=False)

# Guessing the gender for korean names {only english}

In [38]:
df_main_KR_gender_1 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names.csv")
print(df_main_KR_gender_1.shape)
df_main_KR_gender_1.head()

(40007, 8)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year
0,USD0775625,KR,Kim Dong-Hyun,Dong-Hyun,KR,KR101515629B1/en,김동현,2017
1,US09532634,KR,Lee Cheon Sook,Cheon Sook,KR,KR101469765B1/en,이영훈,2017
2,US09532636,KR,Jung Ku Youl,Ku Youl,KR,KR101318467B1/en,정규열,2017
3,US09532637,KR,Choi Jung Sun,Jung Sun,KR,KR20140094485A/en,최정선,2017
4,US09532637,KR,Kim Kyung Nam,Kyung Nam,KR,KR20140094485A/en,김경남,2017


In [39]:
df_main_KR_gender_1["english_gender"] = df_main_KR_gender_1["english_first_names"].apply(english_gender_KR)

In [40]:
df_main_KR_gender_1["english_gender"].value_counts()

unknown          31485
male              5033
female            1889
andy               964
mostly_male        490
mostly_female      146
Name: english_gender, dtype: int64

In [41]:
print(df_main_KR_gender_1.shape)
df_main_KR_gender_1.head()

(40007, 9)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender
0,USD0775625,KR,Kim Dong-Hyun,Dong-Hyun,KR,KR101515629B1/en,김동현,2017,male
1,US09532634,KR,Lee Cheon Sook,Cheon Sook,KR,KR101469765B1/en,이영훈,2017,unknown
2,US09532636,KR,Jung Ku Youl,Ku Youl,KR,KR101318467B1/en,정규열,2017,unknown
3,US09532637,KR,Choi Jung Sun,Jung Sun,KR,KR20140094485A/en,최정선,2017,female
4,US09532637,KR,Kim Kyung Nam,Kyung Nam,KR,KR20140094485A/en,김경남,2017,unknown


In [42]:
df_main_KR_gender_1.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN.csv",index=False)

# Original Korean Names Gender Namsor Function

In [43]:
import ast
import requests
import pandas as pd
import numpy as np

In [48]:
df_main_KR = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN.csv")

In [49]:
print(df_main_KR.shape)
df_main_KR.head()

(40007, 9)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender
0,USD0775625,KR,Kim Dong-Hyun,Dong-Hyun,KR,KR101515629B1/en,김동현,2017,male
1,US09532634,KR,Lee Cheon Sook,Cheon Sook,KR,KR101469765B1/en,이영훈,2017,unknown
2,US09532636,KR,Jung Ku Youl,Ku Youl,KR,KR101318467B1/en,정규열,2017,unknown
3,US09532637,KR,Choi Jung Sun,Jung Sun,KR,KR20140094485A/en,최정선,2017,female
4,US09532637,KR,Kim Kyung Nam,Kyung Nam,KR,KR20140094485A/en,김경남,2017,unknown


In [50]:
df_main_KR["korean_likelyGender"] = np.nan
df_main_KR["korean_genderScale"] = np.nan
df_main_KR["korean_score"] = np.nan
df_main_KR["korean_probabilityCalibrated"] = np.nan

In [55]:
df_main_KR["korean_likelyGender"].count()

0

In [51]:
for i in range(len(df_main_KR.columns)):
  print(i,df_main_KR.columns[i])

0 publication_id
1 country
2 english_names
3 english_first_names
4 inventor_country
5 worldwide_application
6 original_names
7 year
8 english_gender
9 korean_likelyGender
10 korean_genderScale
11 korean_score
12 korean_probabilityCalibrated


In [113]:
def namsor_extraction_korean(df, api_key):

  url = "https://v2.namsor.com/NamSorAPIv2/api2/json/genderBatch"

  headers = {
      "X-API-KEY": api_key,
      "Accept": "application/json",
      "Content-Type": "application/json"
  }
  
  payload = {"personalNames": []}
  #start_index = 0
  last_index = 0

  try:
    already_df = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
    start_point = already_df["korean_likelyGender"].count()
    start_index = already_df["korean_likelyGender"].count()
  except:
    start_point = 0
    start_index = 0
  print(start_point)

  #print(0/0)

  for i in range(start_point,df.shape[0]):

    if len(payload["personalNames"])>=100:

      print(start_index,last_index)
      
      response = requests.request("POST", url, json=payload, headers=headers)
      genders = ast.literal_eval(response.text)

      with open("/content/drive/MyDrive/CS 506/Project/"+str(start_index)+"_KR_Names_Nov_22.json", "w") as outfile:
        json.dump(genders, outfile)

      gend_idx = 0
      
      for k in range(start_index,last_index):

        gend = genders['personalNames'][gend_idx]
        
        df.iloc[k,9] = gend['likelyGender']
        df.iloc[k,10] = gend['genderScale']
        df.iloc[k,11] = gend['score']
        df.iloc[k,12] = gend['probabilityCalibrated']
        
        gend_idx += 1
      
      payload = {"personalNames": []}
      start_index = i

      df.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv",index=False)
      print(df.shape)

    n = df.iloc[i,6]
    d = {"firstName": n[1:],"lastName": n[0]}
    payload["personalNames"].append(d)
    last_index = i+1
  
  if len(payload["personalNames"])>0:

    print(start_index,last_index)
      
    response = requests.request("POST", url, json=payload, headers=headers)
    genders = ast.literal_eval(response.text)

    with open("/content/drive/MyDrive/CS 506/Project/"+str(start_index)+"_KR_Names_Nov_22.json", "w") as outfile:
      json.dump(genders, outfile)

    gend_idx = 0
    
    for k in range(start_index,last_index):

      gend = genders['personalNames'][gend_idx]
      
      df.iloc[k,9] = gend['likelyGender']
      df.iloc[k,10] = gend['genderScale']
      df.iloc[k,11] = gend['score']
      df.iloc[k,12] = gend['probabilityCalibrated']
      
      gend_idx += 1
    
    df.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv",index=False)
  
  return df


In [57]:
df_main_KR_namsor_KR = namsor_extraction_korean(df_main_KR, "c402eb1d686e40fa7c4d6f20600bb2c4")

0
0 100
(40007, 13)
100 200
(40007, 13)
200 300
(40007, 13)
300 400
(40007, 13)
400 500
(40007, 13)
500 600
(40007, 13)
600 700
(40007, 13)
700 800
(40007, 13)
800 900
(40007, 13)
900 1000
(40007, 13)
1000 1100
(40007, 13)
1100 1200
(40007, 13)
1200 1300
(40007, 13)
1300 1400
(40007, 13)
1400 1500
(40007, 13)
1500 1600
(40007, 13)
1600 1700
(40007, 13)
1700 1800
(40007, 13)
1800 1900
(40007, 13)
1900 2000
(40007, 13)
2000 2100
(40007, 13)
2100 2200
(40007, 13)
2200 2300
(40007, 13)
2300 2400
(40007, 13)
2400 2500
(40007, 13)
2500 2600
(40007, 13)
2600 2700
(40007, 13)
2700 2800
(40007, 13)
2800 2900
(40007, 13)
2900 3000
(40007, 13)
3000 3100
(40007, 13)
3100 3200
(40007, 13)
3200 3300
(40007, 13)
3300 3400
(40007, 13)
3400 3500
(40007, 13)
3500 3600
(40007, 13)
3600 3700
(40007, 13)
3700 3800
(40007, 13)
3800 3900
(40007, 13)
3900 4000
(40007, 13)
4000 4100
(40007, 13)
4100 4200
(40007, 13)
4200 4300
(40007, 13)
4300 4400
(40007, 13)
4400 4500
(40007, 13)
4500 4600
(40007, 13)
4600 47

SyntaxError: ignored

In [86]:
already_df_3 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")

In [87]:
df_main_KR_namsor_KR = namsor_extraction_korean(already_df_3 , "c9b13d67a29a4943f5a583fe86654131")

10200
10200 10300
(40007, 13)
10300 10400
(40007, 13)
10400 10500
(40007, 13)
10500 10600
(40007, 13)
10600 10700
(40007, 13)
10700 10800
(40007, 13)
10800 10900
(40007, 13)
10900 11000
(40007, 13)
11000 11100
(40007, 13)
11100 11200
(40007, 13)
11200 11300
(40007, 13)
11300 11400
(40007, 13)
11400 11500
(40007, 13)
11500 11600
(40007, 13)
11600 11700
(40007, 13)
11700 11800
(40007, 13)
11800 11900
(40007, 13)
11900 12000
(40007, 13)
12000 12100
(40007, 13)
12100 12200
(40007, 13)
12200 12300
(40007, 13)
12300 12400
(40007, 13)
12400 12500
(40007, 13)
12500 12600
(40007, 13)
12600 12700
(40007, 13)
12700 12800
(40007, 13)
12800 12900
(40007, 13)
12900 13000
(40007, 13)
13000 13100
(40007, 13)
13100 13200
(40007, 13)
13200 13300
(40007, 13)
13300 13400
(40007, 13)
13400 13500
(40007, 13)
13500 13600
(40007, 13)
13600 13700
(40007, 13)
13700 13800
(40007, 13)
13800 13900
(40007, 13)
13900 14000
(40007, 13)
14000 14100
(40007, 13)
14100 14200
(40007, 13)
14200 14300
(40007, 13)
14300 1440

SyntaxError: ignored

In [94]:
already_df_5 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")

In [95]:
df_main_KR_namsor_KR = namsor_extraction_korean(already_df_5 , "495a53e75c056ed3f862b3a6424646a0")

18200
18200 18300
(40007, 13)
18300 18400
(40007, 13)
18400 18500
(40007, 13)
18500 18600
(40007, 13)
18600 18700
(40007, 13)
18700 18800
(40007, 13)
18800 18900
(40007, 13)
18900 19000
(40007, 13)
19000 19100
(40007, 13)
19100 19200
(40007, 13)
19200 19300
(40007, 13)
19300 19400
(40007, 13)
19400 19500
(40007, 13)
19500 19600
(40007, 13)
19600 19700
(40007, 13)
19700 19800
(40007, 13)
19800 19900
(40007, 13)
19900 20000
(40007, 13)
20000 20100
(40007, 13)
20100 20200
(40007, 13)
20200 20300
(40007, 13)
20300 20400
(40007, 13)
20400 20500
(40007, 13)
20500 20600
(40007, 13)
20600 20700
(40007, 13)
20700 20800
(40007, 13)
20800 20900
(40007, 13)
20900 21000
(40007, 13)
21000 21100
(40007, 13)
21100 21200
(40007, 13)
21200 21300
(40007, 13)
21300 21400
(40007, 13)
21400 21500
(40007, 13)
21500 21600
(40007, 13)
21600 21700
(40007, 13)
21700 21800
(40007, 13)
21800 21900
(40007, 13)
21900 22000
(40007, 13)
22000 22100
(40007, 13)
22100 22200
(40007, 13)
22200 22300
(40007, 13)
22300 2240

SyntaxError: ignored

In [103]:
already_df_7 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")

In [104]:

df_main_KR_namsor_KR = namsor_extraction_korean(already_df_7 , "f327fcaf129c742d0d1874a05f13a833")

26200
26200 26300
(40007, 13)
26300 26400
(40007, 13)
26400 26500
(40007, 13)
26500 26600
(40007, 13)
26600 26700
(40007, 13)
26700 26800
(40007, 13)
26800 26900
(40007, 13)
26900 27000
(40007, 13)
27000 27100
(40007, 13)
27100 27200
(40007, 13)
27200 27300
(40007, 13)
27300 27400
(40007, 13)
27400 27500
(40007, 13)
27500 27600
(40007, 13)
27600 27700
(40007, 13)
27700 27800
(40007, 13)
27800 27900
(40007, 13)
27900 28000
(40007, 13)
28000 28100
(40007, 13)
28100 28200
(40007, 13)
28200 28300
(40007, 13)
28300 28400
(40007, 13)
28400 28500
(40007, 13)
28500 28600
(40007, 13)
28600 28700
(40007, 13)
28700 28800
(40007, 13)
28800 28900
(40007, 13)
28900 29000
(40007, 13)
29000 29100
(40007, 13)
29100 29200
(40007, 13)
29200 29300
(40007, 13)
29300 29400
(40007, 13)
29400 29500
(40007, 13)
29500 29600
(40007, 13)
29600 29700
(40007, 13)
29700 29800
(40007, 13)
29800 29900
(40007, 13)
29900 30000
(40007, 13)
30000 30100
(40007, 13)
30100 30200
(40007, 13)
30200 30300
(40007, 13)
30300 3040

SyntaxError: ignored

In [114]:
already_df_9 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")

In [115]:
df_main_KR_namsor_KR = namsor_extraction_korean(already_df_9 , "9be3227b70239ee6be1ecb30d283ce5e")

34500
34500 34600
(40007, 13)
34600 34700
(40007, 13)
34700 34800
(40007, 13)
34800 34900
(40007, 13)
34900 35000
(40007, 13)
35000 35100
(40007, 13)
35100 35200
(40007, 13)
35200 35300
(40007, 13)
35300 35400
(40007, 13)
35400 35500
(40007, 13)
35500 35600
(40007, 13)
35600 35700
(40007, 13)
35700 35800
(40007, 13)
35800 35900
(40007, 13)
35900 36000
(40007, 13)
36000 36100
(40007, 13)
36100 36200
(40007, 13)
36200 36300
(40007, 13)
36300 36400
(40007, 13)
36400 36500
(40007, 13)
36500 36600
(40007, 13)
36600 36700
(40007, 13)
36700 36800
(40007, 13)
36800 36900
(40007, 13)
36900 37000
(40007, 13)
37000 37100
(40007, 13)
37100 37200
(40007, 13)
37200 37300
(40007, 13)
37300 37400
(40007, 13)
37400 37500
(40007, 13)
37500 37600
(40007, 13)
37600 37700
(40007, 13)
37700 37800
(40007, 13)
37800 37900
(40007, 13)
37900 38000
(40007, 13)
38000 38100
(40007, 13)
38100 38200
(40007, 13)
38200 38300
(40007, 13)
38300 38400
(40007, 13)
38400 38500
(40007, 13)
38500 38600
(40007, 13)
38600 3870

In [ ]:
print(df_main_KR_namsor_KR.shape)
df_main_KR_namsor_KR.head()

In [ ]:
#df_main_KR_namsor_KR.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv",index=False)

In [58]:
already_df = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
start_point = already_df["korean_likelyGender"].count()
start_point

10200

In [61]:
already_df.iloc[10195:10205,:]

,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
10195,US09865874,KR,Park Hong Kyu,Hong Kyu,KR,KR101565298B1/en,박홍규,2018,male,-0.994915,41.421940,0.997457,NaN
10196,US09865874,KR,Oh Sang Seung,Sang Seung,KR,KR101565298B1/en,오상승,2018,male,-0.931456,18.196450,0.965728,NaN
10197,US09865874,KR,Jo Chi Ho,Chi Ho,KR,KR101565298B1/en,조치호,2018,male,-0.985249,25.377543,0.992625,NaN
10198,US09865875,KR,Jun In Kook,In Kook,KR,KR101561376B1/en,전인국,2018,male,-0.991397,29.448981,0.995699,NaN
10199,US09865875,KR,Cho Seung Beom,Seung Beom,KR,KR101561376B1/en,조승범,2018,male,-0.993166,32.123802,0.996583,NaN
10200,US09865875,KR,Oh Myoung Hwan,Myoung Hwan,KR,KR101561376B1/en,오명환,2018,unknown,NaN,NaN,NaN,NaN
10201,US09865877,KR,Kang Yong-Mook,Yong-Mook,KR,KR101673300B1/en,강용묵,2018,unknown,NaN,NaN,NaN,NaN
10202,US09865877,KR,Yang Jung-Hoon,Jung-Hoon,KR,KR101673300B1/en,양정훈,2018,male,NaN,NaN,NaN,NaN
10203,US09865888,KR,Kwon Sang Uk,Sang Uk,KR,KR101628514B1/en,권상욱,2018,unknown,NaN,NaN,NaN,NaN
10204,US09865891,KR,Lee Dong Hun,Dong Hun,KR,KR101846625B1/en,이동훈,2018,male,NaN,NaN,NaN,NaN


In [62]:
already_df.iloc[:,-1] = already_df.iloc[:,-2]

In [63]:
already_df.iloc[10195:10205,:]

,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
10195,US09865874,KR,Park Hong Kyu,Hong Kyu,KR,KR101565298B1/en,박홍규,2018,male,-0.994915,41.421940,0.997457,0.997457
10196,US09865874,KR,Oh Sang Seung,Sang Seung,KR,KR101565298B1/en,오상승,2018,male,-0.931456,18.196450,0.965728,0.965728
10197,US09865874,KR,Jo Chi Ho,Chi Ho,KR,KR101565298B1/en,조치호,2018,male,-0.985249,25.377543,0.992625,0.992625
10198,US09865875,KR,Jun In Kook,In Kook,KR,KR101561376B1/en,전인국,2018,male,-0.991397,29.448981,0.995699,0.995699
10199,US09865875,KR,Cho Seung Beom,Seung Beom,KR,KR101561376B1/en,조승범,2018,male,-0.993166,32.123802,0.996583,0.996583
10200,US09865875,KR,Oh Myoung Hwan,Myoung Hwan,KR,KR101561376B1/en,오명환,2018,unknown,NaN,NaN,NaN,NaN
10201,US09865877,KR,Kang Yong-Mook,Yong-Mook,KR,KR101673300B1/en,강용묵,2018,unknown,NaN,NaN,NaN,NaN
10202,US09865877,KR,Yang Jung-Hoon,Jung-Hoon,KR,KR101673300B1/en,양정훈,2018,male,NaN,NaN,NaN,NaN
10203,US09865888,KR,Kwon Sang Uk,Sang Uk,KR,KR101628514B1/en,권상욱,2018,unknown,NaN,NaN,NaN,NaN
10204,US09865891,KR,Lee Dong Hun,Dong Hun,KR,KR101846625B1/en,이동훈,2018,male,NaN,NaN,NaN,NaN


In [64]:
already_df.iloc[:,-2] = already_df.iloc[:,-3]
already_df.iloc[:,-3] = already_df.iloc[:,-4]
already_df.iloc[:,-4] = already_df.iloc[:,-5]

In [70]:
already_df.iloc[10195:10205,:]

,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
10195,US09865874,KR,Park Hong Kyu,Hong Kyu,KR,KR101565298B1/en,박홍규,2018.0,male,male,-0.994915,41.421940,0.997457
10196,US09865874,KR,Oh Sang Seung,Sang Seung,KR,KR101565298B1/en,오상승,2018.0,male,male,-0.931456,18.196450,0.965728
10197,US09865874,KR,Jo Chi Ho,Chi Ho,KR,KR101565298B1/en,조치호,2018.0,male,male,-0.985249,25.377543,0.992625
10198,US09865875,KR,Jun In Kook,In Kook,KR,KR101561376B1/en,전인국,2018.0,male,male,-0.991397,29.448981,0.995699
10199,US09865875,KR,Cho Seung Beom,Seung Beom,KR,KR101561376B1/en,조승범,2018.0,male,male,-0.993166,32.123802,0.996583
10200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
already_df.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv",index=False)

In [67]:
already_df_2 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
start_point = already_df_2["korean_likelyGender"].count()
start_point

40007

In [71]:
already_df_2.iloc[10195:10205,:]

,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
10195,US09865874,KR,Park Hong Kyu,Hong Kyu,KR,KR101565298B1/en,박홍규,2018,male,male,-0.994915,41.421940,0.997457
10196,US09865874,KR,Oh Sang Seung,Sang Seung,KR,KR101565298B1/en,오상승,2018,male,male,-0.931456,18.196450,0.965728
10197,US09865874,KR,Jo Chi Ho,Chi Ho,KR,KR101565298B1/en,조치호,2018,male,male,-0.985249,25.377543,0.992625
10198,US09865875,KR,Jun In Kook,In Kook,KR,KR101561376B1/en,전인국,2018,male,male,-0.991397,29.448981,0.995699
10199,US09865875,KR,Cho Seung Beom,Seung Beom,KR,KR101561376B1/en,조승범,2018,male,male,-0.993166,32.123802,0.996583
10200,US09865875,KR,Oh Myoung Hwan,Myoung Hwan,KR,KR101561376B1/en,오명환,2018,unknown,unknown,NaN,NaN,NaN
10201,US09865877,KR,Kang Yong-Mook,Yong-Mook,KR,KR101673300B1/en,강용묵,2018,unknown,unknown,NaN,NaN,NaN
10202,US09865877,KR,Yang Jung-Hoon,Jung-Hoon,KR,KR101673300B1/en,양정훈,2018,male,male,NaN,NaN,NaN
10203,US09865888,KR,Kwon Sang Uk,Sang Uk,KR,KR101628514B1/en,권상욱,2018,unknown,unknown,NaN,NaN,NaN
10204,US09865891,KR,Lee Dong Hun,Dong Hun,KR,KR101846625B1/en,이동훈,2018,male,male,NaN,NaN,NaN


In [72]:
already_df_2.iloc[10200:,-4] = np.nan

In [73]:
already_df_2.iloc[10195:10205,:]

,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
10195,US09865874,KR,Park Hong Kyu,Hong Kyu,KR,KR101565298B1/en,박홍규,2018,male,male,-0.994915,41.421940,0.997457
10196,US09865874,KR,Oh Sang Seung,Sang Seung,KR,KR101565298B1/en,오상승,2018,male,male,-0.931456,18.196450,0.965728
10197,US09865874,KR,Jo Chi Ho,Chi Ho,KR,KR101565298B1/en,조치호,2018,male,male,-0.985249,25.377543,0.992625
10198,US09865875,KR,Jun In Kook,In Kook,KR,KR101561376B1/en,전인국,2018,male,male,-0.991397,29.448981,0.995699
10199,US09865875,KR,Cho Seung Beom,Seung Beom,KR,KR101561376B1/en,조승범,2018,male,male,-0.993166,32.123802,0.996583
10200,US09865875,KR,Oh Myoung Hwan,Myoung Hwan,KR,KR101561376B1/en,오명환,2018,unknown,NaN,NaN,NaN,NaN
10201,US09865877,KR,Kang Yong-Mook,Yong-Mook,KR,KR101673300B1/en,강용묵,2018,unknown,NaN,NaN,NaN,NaN
10202,US09865877,KR,Yang Jung-Hoon,Jung-Hoon,KR,KR101673300B1/en,양정훈,2018,male,NaN,NaN,NaN,NaN
10203,US09865888,KR,Kwon Sang Uk,Sang Uk,KR,KR101628514B1/en,권상욱,2018,unknown,NaN,NaN,NaN,NaN
10204,US09865891,KR,Lee Dong Hun,Dong Hun,KR,KR101846625B1/en,이동훈,2018,male,NaN,NaN,NaN,NaN


In [74]:
start_point = already_df_2["korean_likelyGender"].count()
start_point

10200

In [75]:
already_df_2.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv",index=False)

In [88]:
already_df_4 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
start_point = already_df_4["korean_likelyGender"].count()
start_point

18200

In [89]:
print(already_df_4.shape)
already_df_4.iloc[start_point-5:start_point+5,:]

(40007, 13)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
18195,US10177380,KR,Jung Dong Sub,Dong Sub,KR,KR101790400B1/en,정동섭,2019,unknown,male,-0.994915,44.524427,0.997457
18196,US10177380,KR,Kim Eun Kyung,Eun Kyung,KR,KR101790400B1/en,김은경,2019,female,female,0.981326,23.917801,0.990663
18197,US10177380,KR,Woo Sang Wook,Sang Wook,KR,KR101790400B1/en,우상욱,2019,male,male,-0.987953,27.064740,0.993976
18198,US10177380,KR,Shin Sun Young,Sun Young,KR,KR101790400B1/en,신선영,2019,female,female,0.784704,10.939312,0.892352
18199,US10177384,US,Chu Hee-Young,Hee-Young,KR,KR101775543B1/en,추희영,2019,mostly_female,male,-0.051987,2.418506,0.525994
18200,US10177384,US,Moon Sung-Hwan,Sung-Hwan,KR,KR101775543B1/en,문성환,2019,male,NaN,NaN,NaN,NaN
18201,US10177384,US,Kim Jae-Hyuk,Jae-Hyuk,KR,KR101775543B1/en,김재혁,2019,unknown,NaN,NaN,NaN,NaN
18202,US10177384,US,Jeong Myung-Hwan,Myung-Hwan,KR,KR101775543B1/en,정명환,2019,unknown,NaN,NaN,NaN,NaN
18203,US10177384,US,Jeong Chang-Ui,Chang-Ui,KR,KR101775543B1/en,정창의,2019,unknown,NaN,NaN,NaN,NaN
18204,US10177384,US,Choi Jong-Seo,Jong-Seo,KR,KR101775543B1/en,최종서,2019,unknown,NaN,NaN,NaN,NaN


In [96]:
already_df_6 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
start_point = already_df_6["korean_likelyGender"].count()
start_point

26200

In [97]:
print(already_df_6.shape)
already_df_6.iloc[start_point-5:start_point+5,:]

(40007, 13)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
26195,US10536573,KR,Kong Daejin,Daejin,KR,KR20170006558A/en,공대진,2020,unknown,male,-0.973499,21.721952,0.986749
26196,US10536573,KR,Yi Suhjin,Suhjin,KR,KR20170006558A/en,이서진,2020,unknown,male,-0.032051,2.280282,0.516025
26197,US10536573,KR,Jung Juhyun,Juhyun,KR,KR20170006558A/en,정주현,2020,unknown,male,-0.790971,11.126664,0.895486
26198,US10536573,KR,Jang Yujune,Yujune,KR,KR20170006558A/en,장유준,2020,unknown,male,-0.817655,13.224716,0.908827
26199,US10536662,KR,Lee Hyeon-ji,Hyeon-ji,KR,KR20120023420A/en,이현지,2020,unknown,female,0.277024,5.730573,0.638512
26200,US10536662,KR,Kwon Yong-hwan,Yong-hwan,KR,KR20120023420A/en,권용환,2020,unknown,NaN,NaN,NaN,NaN
26201,US10536662,KR,Ahn Jun-mo,Jun-mo,KR,KR20120023420A/en,안준모,2020,unknown,NaN,NaN,NaN,NaN
26202,US10536662,KR,Bae Hee-jeong,Hee-jeong,KR,KR20120023420A/en,배희정,2020,unknown,NaN,NaN,NaN,NaN
26203,US10536697,KR,Choi Dai-woong,Dai-woong,KR,KR101703330B1/en,최대웅,2020,unknown,NaN,NaN,NaN,NaN
26204,US10536697,KR,Choi Woong-il,Woong-il,KR,KR101703330B1/en,최웅일,2020,unknown,NaN,NaN,NaN,NaN


In [105]:
already_df_8 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
start_point = already_df_8["korean_likelyGender"].count()
start_point

34500

In [106]:
print(already_df_8.shape)
already_df_8.iloc[start_point-5:start_point+5,:]

(40007, 13)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
34495,US10894843,KR,Shin Eunji,Eunji,KR,KR102133030B1/en,신은지,2021,unknown,female,0.983670,24.666410,0.991835
34496,US10894843,KR,Lee Ki Soo,Ki Soo,KR,KR102133030B1/en,이기수,2021,male,male,-0.991937,29.911545,0.995969
34497,US10894843,KR,Hong Daesik,Daesik,KR,KR102133030B1/en,홍대식,2021,unknown,male,-0.994863,35.438416,0.997431
34498,US10894843,KR,Park Sung Ho,Sung Ho,KR,KR102133030B1/en,박성호,2021,male,male,-0.994863,35.552498,0.997431
34499,US10894843,KR,You Youngsuk,Youngsuk,KR,KR102133030B1/en,유영석,2021,unknown,male,-0.989853,28.131266,0.994927
34500,US10894843,KR,Lee Jinyoung,Jinyoung,KR,KR102133030B1/en,이진영,2021,unknown,NaN,NaN,NaN,NaN
34501,US10894844,KR,Lee Yong Hun,Yong Hun,KR,KR20190012811A/en,이용훈,2021,unknown,NaN,NaN,NaN,NaN
34502,US10894844,KR,Won Tae Young,Tae Young,KR,KR20190012811A/en,원태영,2021,male,NaN,NaN,NaN,NaN
34503,US10894844,KR,Lee Jun Wye,Jun Wye,KR,KR20190012811A/en,이준의,2021,unknown,NaN,NaN,NaN,NaN
34504,US10894844,KR,Shin Kwangin,Kwangin,KR,KR20190012811A/en,신광인,2021,unknown,NaN,NaN,NaN,NaN


In [116]:
already_df_10 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
start_point = already_df_10["korean_likelyGender"].count()
start_point

40007

In [117]:
print(already_df_10.shape)
already_df_10.iloc[start_point-5:start_point+5,:]

(40007, 13)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
40002,US10921294,KR,Kim Seok Jin,Seok Jin,KR,KR102141380B1/en,김석진,2021,unknown,male,-0.991046,28.965042,0.995523
40003,US10921294,KR,Ku Cha Hun,Cha Hun,KR,KR102141380B1/en,구자훈,2021,unknown,male,-0.809226,12.043413,0.904613
40004,US10950523,KR,Kim Dong-Wan,Dong-Wan,KR,KR20170104045A/en,김동완,2021,male,male,-0.994880,36.095765,0.997440
40005,US10950523,KR,Han Jung-Hoon,Jung-Hoon,KR,KR20170104045A/en,한정훈,2021,male,male,-0.994620,33.921702,0.997310
40006,US10950523,KR,Park Dong-Sik,Dong-Sik,KR,KR20170104045A/en,박동식,2021,male,male,-0.994915,42.666423,0.997457


# Run english on korean again.

In [118]:
df_main_KR_gender_1 = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_KR.csv")
print(df_main_KR_gender_1.shape)
df_main_KR_gender_1.head()


(40007, 13)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
0,USD0775625,KR,Kim Dong-Hyun,Dong-Hyun,KR,KR101515629B1/en,김동현,2017,male,male,-0.994915,38.123614,0.997457
1,US09532634,KR,Lee Cheon Sook,Cheon Sook,KR,KR101469765B1/en,이영훈,2017,male,male,-0.992919,31.619904,0.996460
2,US09532636,KR,Jung Ku Youl,Ku Youl,KR,KR101318467B1/en,정규열,2017,male,male,-0.988202,27.418114,0.994101
3,US09532637,KR,Choi Jung Sun,Jung Sun,KR,KR20140094485A/en,최정선,2017,male,male,-0.405357,7.265855,0.702679
4,US09532637,KR,Kim Kyung Nam,Kyung Nam,KR,KR20140094485A/en,김경남,2017,male,male,-0.892104,16.558482,0.946052


In [119]:
df_main_KR_gender_1["english_gender"] = df_main_KR_gender_1["english_first_names"].apply(english_gender_KR)

In [120]:
df_main_KR_gender_1["english_gender"].value_counts()

unknown          31485
male              5033
female            1889
andy               964
mostly_male        490
mostly_female      146
Name: english_gender, dtype: int64

In [121]:
print(df_main_KR_gender_1.shape)
df_main_KR_gender_1.head()


(40007, 13)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
0,USD0775625,KR,Kim Dong-Hyun,Dong-Hyun,KR,KR101515629B1/en,김동현,2017,male,male,-0.994915,38.123614,0.997457
1,US09532634,KR,Lee Cheon Sook,Cheon Sook,KR,KR101469765B1/en,이영훈,2017,unknown,male,-0.992919,31.619904,0.996460
2,US09532636,KR,Jung Ku Youl,Ku Youl,KR,KR101318467B1/en,정규열,2017,unknown,male,-0.988202,27.418114,0.994101
3,US09532637,KR,Choi Jung Sun,Jung Sun,KR,KR20140094485A/en,최정선,2017,female,male,-0.405357,7.265855,0.702679
4,US09532637,KR,Kim Kyung Nam,Kyung Nam,KR,KR20140094485A/en,김경남,2017,unknown,male,-0.892104,16.558482,0.946052


In [122]:
df_main_KR_gender_1.to_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_2_KR.csv",index=False)

# Visualizations

In [123]:
df_chinese = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_CN_Names_Gender_CN_EN.csv")
print(df_chinese.shape)
df_chinese.head()

(40017, 11)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,chinese_gender,chinese_gender_prob
0,US09532512,CN,Xie Yunlong,Yunlong,CN,CN104823754A/en,谢云龙,2017,unknown,M,0.945586
1,US09532598,US,Liu Qiuming,Qiuming,CN,CN205214206U/en,刘秋明,2017,unknown,M,0.500785
2,US09532599,CN,Liu Qiuming,Qiuming,CN,CN203327956U/en,刘秋明,2017,unknown,M,0.500785
3,US09532602,CN,Liu Qiuming,Qiuming,CN,CN203378559U/en,刘秋明,2017,unknown,M,0.500785
4,US09532631,WO,Gu Jiawei,Jiawei,CN,CN105378583B/en,顾嘉唯,2017,unknown,M,0.496414


In [125]:
df_chinese["chinese_gender"].value_counts()

M                31866
F                 7208
Undefined          887
M=F=Undefined       56
Name: chinese_gender, dtype: int64

In [126]:
df_chinese["english_gender"].value_counts()

unknown          24683
andy             12471
male              1467
mostly_male        849
female             443
mostly_female      104
Name: english_gender, dtype: int64

In [124]:
df_korean = pd.read_csv("/content/drive/MyDrive/Team 1/Data/Team_1_Original_Names/Nov_22_KR_Names_Gender_EN_2_KR.csv")
print(df_korean.shape)
df_korean.head()

(40007, 13)


,publication_id,country,english_names,english_first_names,inventor_country,worldwide_application,original_names,year,english_gender,korean_likelyGender,korean_genderScale,korean_score,korean_probabilityCalibrated
0,USD0775625,KR,Kim Dong-Hyun,Dong-Hyun,KR,KR101515629B1/en,김동현,2017,male,male,-0.994915,38.123614,0.997457
1,US09532634,KR,Lee Cheon Sook,Cheon Sook,KR,KR101469765B1/en,이영훈,2017,unknown,male,-0.992919,31.619904,0.996460
2,US09532636,KR,Jung Ku Youl,Ku Youl,KR,KR101318467B1/en,정규열,2017,unknown,male,-0.988202,27.418114,0.994101
3,US09532637,KR,Choi Jung Sun,Jung Sun,KR,KR20140094485A/en,최정선,2017,female,male,-0.405357,7.265855,0.702679
4,US09532637,KR,Kim Kyung Nam,Kyung Nam,KR,KR20140094485A/en,김경남,2017,unknown,male,-0.892104,16.558482,0.946052


In [127]:
df_korean["english_gender"].value_counts()

unknown          31485
male              5033
female            1889
andy               964
mostly_male        490
mostly_female      146
Name: english_gender, dtype: int64

In [128]:
df_korean["korean_likelyGender"].value_counts()

male      36232
female     3775
Name: korean_likelyGender, dtype: int64

In [130]:
df_korean["korean_probabilityCalibrated"].value_counts()

0.997457    3752
0.997449     832
0.997440     682
0.997431     467
0.997423     323
            ... 
0.981467       1
0.991721       1
0.979671       1
0.610436       1
0.991218       1
Name: korean_probabilityCalibrated, Length: 12447, dtype: int64